In [9]:
import tweepy 
import time 
import pandas as pd
import time

#put bearer troken here
bearertoken = 'Your super secret bearer token here'
#Declaring the client auth
client = tweepy.Client(bearer_token=bearertoken,wait_on_rate_limit=True)

#Based on my data analysis for a symposium presentation, I know this query should return roughly 22000 tweets (based on the results fromm the GUI tweet downloader)
query = 'tigerselfie OR tigercubselfie'
start_time = '2012-01-01T00:00:00Z'
end_time = '2022-12-31T00:00:00Z'


formatted_tweets = []
for tweets in tweepy.Paginator(client.search_all_tweets,
                                   query=query,
                                   max_results = 100,
                                   start_time=start_time,
                                   end_time=end_time,
                                   expansions=['attachments.poll_ids', 'attachments.media_keys', 'author_id',
                                               'entities.mentions.username', 'geo.place_id', 'in_reply_to_user_id',
                                               'referenced_tweets.id', 'referenced_tweets.id.author_id'],
                                   media_fields=['duration_ms', 'height', 'media_key', 'preview_image_url', 'type',
                                                 'url', 'width', 'public_metrics', 'alt_text'],
                                   place_fields=['contained_within', 'country', 'country_code', 'full_name', 'geo',
                                                 'id', 'name', 'place_type'],
                                   poll_fields=['duration_minutes', 'end_datetime', 'id', 'options', 'voting_status'],
                                
                                   tweet_fields=['attachments', 'author_id', 'context_annotations', 'conversation_id',
                                                 'created_at', 'entities', 'geo', 'id', 'in_reply_to_user_id', 'lang',
                                                 'public_metrics', 'possibly_sensitive', 'referenced_tweets',
                                                 'reply_settings', 'source', 'text', 'withheld'],
                                   user_fields=['created_at', 'description', 'entities', 'id', 'location', 'name',
                                                'pinned_tweet_id', 'profile_image_url', 'protected', 'public_metrics',
                                                'url', 'username', 'verified', 'withheld'],
                                   ):
    # Gets list of places from includes object
    places = None
    users = None
    media = None
    
    if bool(tweets.includes):
        if "places" in tweets.includes:
            places = {p['id']: p for p in tweets.includes['places']}

    # Gets users list from the includes object
        if "users" in tweets.includes:
            users = {u["id"]: u for u in tweets.includes['users']}

    #Need to add code that created media dict here 
        if "media" in tweets.includes:
            media = {m["media_key"]: m for m in tweets.includes['media']}


    if bool(tweets.data):
        for tweet in tweets.data:
            formatted_tweet = {
                "tweet_id": tweet.id,
                "tweet_text": tweet.text,
                "tweet_created_at": tweet.created_at,
                "author_id": tweet.author_id,
                "conversation_id": tweet.conversation_id,
                "in_reply_to_user_id": tweet.in_reply_to_user_id,  # Output: None or Tweet ID
                "referenced_tweets": tweet['referenced_tweets'],  # Output: None or XML Nodes inside JSON Array
                "attachments": tweet.attachments,  # Ouput: None or JSON Array
                "tweet_geo": tweet.geo,
                # Output: None or Just place_id or place_id and coordinates with coordinates id and actual coordinates
                "context_annotations": tweet.context_annotations,
                # Output: An empty array or JSON arrays of arrays with domain and entity along with their own JSON arrays
                "entities": tweet.entities,
                # Output: None or JSON array with Corresponding Keys who have thier own arrays. Many keys could be missing or included.
                "withheld": tweet.withheld,  # Output: None or array (Did not observe arrays yet)
                "public_metrics": tweet.public_metrics,
                # Output: A JSON array with keys. Did not observe None or empty array
                "possibly_sensitive": tweet.possibly_sensitive,
                # Output: False or True. This field looks promising. Find out more.
                "tweet_language": tweet.lang,  # Output: Could return 'und'. Find out what it means.
                "reply_settings": tweet.reply_settings,
                "tweet_source": tweet.source,
                # Output: Could reveal the device they are using or be Untapped. Find out what Untapped means
            }

            if bool(users):
                if bool(tweet.author_id):
                    user = users[tweet.author_id]
                            
                    for key, value in user.items():
                        formatted_tweet["user_"+str(key)] = value

                
                user = users[tweet.author_id]
                formatted_tweet["user_name"] = user.name,
                formatted_tweet["user_profile_image_url"] =  user.profile_image_url,
                formatted_tweet["user_pinned_tweet_id"] = user.pinned_tweet_id,
                formatted_tweet["user_created_at"] = user.created_at,
                formatted_tweet["user_description"] = user.description,
                formatted_tweet["user_entities"] = user.entities,
                formatted_tweet["user_id"] = user.id,
                formatted_tweet["user_location"] = user.location,
                formatted_tweet["user_protected"] = user.protected,
                formatted_tweet["user_public_metrics"] = user.public_metrics,
                formatted_tweet["user_url"] = user.url,
                formatted_tweet["user_withheld"] = user.withheld,
                formatted_tweet["user_verified"] = user.verified,
                
            if bool(places):
                if bool(tweet.geo):
                    if "place_id" in tweet.geo.keys():
                        place = places[tweet.geo["place_id"]]
                        
                        
                        for key, value in place.items():
                            formatted_tweet["place_"+str(key)] = value
                        
                        formatted_tweet["place_full_name"] = place.full_name,
                        formatted_tweet["place_contained_within"] = place.contained_within,
                        formatted_tweet["place_country_code"] = place.country_code,
                        formatted_tweet["place_geo"] = place.geo,
                        formatted_tweet["place_id"] = place.id,
                        formatted_tweet["place_type"] = place.place_type
            
            if bool(media):
                if "attachments" in tweet.keys():
                    attachments = tweet['attachments']
                    if bool(attachments):
                        if "media_keys" in attachments.keys():
                            media_keys = attachments['media_keys']
                            if bool(media_keys):
                                if len(media_keys) > 0:
                                    if media_keys[0] in media.keys():
                                        if "preview_image_url" in media[media_keys[0]]:
                                            if media[media_keys[0]].preview_image_url:
                                                formatted_tweet["media"] = media[media_keys[0]].preview_image_url


            formatted_tweets.append(formatted_tweet)
    
        time.sleep(2)

In [10]:
tweets_dataframe = pd.DataFrame(formatted_tweets)
tweets_dataframe.shape

(2479, 41)